In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [98]:
df=pd.read_csv("Data/tweet_emotions.csv",sep=",")
df.head(2)

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...


In [99]:
df=df.drop(columns=["tweet_id"])

In [100]:
df.head(3)

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...


In [101]:
df["sentiment"].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [102]:
final_df=df[df['sentiment'].isin(['sadness','happiness','neutral'])]

In [103]:
final_df.head(5)

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
4,neutral,@dannycastillo We want to trade with someone w...
6,sadness,"I should be sleep, but im not! thinking about ..."
8,sadness,@charviray Charlene my love. I miss you


In [104]:
final_df.isna().sum()

sentiment    0
content      0
dtype: int64

In [105]:
final_df["sentiment"] = final_df["sentiment"].replace({
    "sadness": 0,
    "neutral": 1,
    "happiness": 2
})


C:\Users\Prajwal B\AppData\Local\Temp\ipykernel_21396\215703747.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df["sentiment"] = final_df["sentiment"].replace({
C:\Users\Prajwal B\AppData\Local\Temp\ipykernel_21396\215703747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df["sentiment"] = final_df["sentiment"].replace({


In [106]:
final_df

,sentiment,content
1,0,Layin n bed with a headache ughhhh...waitin o...
2,0,Funeral ceremony...gloomy friday...
4,1,@dannycastillo We want to trade with someone w...
6,0,"I should be sleep, but im not! thinking about ..."
8,0,@charviray Charlene my love. I miss you
...,...,...
39992,1,@jasimmo Ooo showing of your French skills!! l...
39993,1,"@sendsome2me haha, yeah. Twitter has many uses..."
39994,2,Succesfully following Tayla!!
39995,1,@JohnLloydTaylor


In [107]:
train,test=train_test_split(final_df,test_size=0.3,random_state=42)

In [108]:
import nltk
import re

In [109]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def lemmatization(text):
    lemmatizer=WordNetLemmatizer()
    text=text.split()
    text=[lemmatizer.lemmatize(y) for y in text]
    return " ".join(text)
def stopword(text):
    stop_words = stopwords.words("english")
    words = [i for i in str(text).split() if i not in stop_words]
    return " ".join(words)
def removing_numbers(text):
    return " ".join([i for i in text.split() if not i.isdigit()])

def lower_text(text):
    return text.lower()


def removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"""), ' ', text)
    text = text.replace("'", "")
    
    ## Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text = " ".join(text.split())
    return text.strip()


def removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def normalize_text(df):
    df.content = df.content.apply(lambda content: lower_text(content))
    df.content = df.content.apply(lambda content: stopword(content))
    df.content = df.content.apply(lambda content: removing_numbers(content))
    df.content = df.content.apply(lambda content: removing_punctuations(content))
    df.content = df.content.apply(lambda content: removing_urls(content))
    df.content = df.content.apply(lambda content: lemmatization(content))
    return df


def normalized_sentence(sentence):
    sentence = lower_text(sentence)
    sentence = stopword(sentence)
    sentence = removing_numbers(sentence)
    sentence = removing_punctuations(sentence)
    sentence = removing_urls(sentence)
    sentence = lemmatization(sentence)
    return sentence


In [ ]:
normalized_sentence("i am good boy , with extra level goodness")

'good boy extra level goodness'

In [112]:
train_data=normalize_text(train)
test_data=normalize_text(test)

In [113]:
test_data

,sentiment,content
26122,1,got home audition awhile ago think went pretty...
23017,2,cant wait thursday
28728,1,tccrt3r heyy rayleneornelas told follow u whatsup
3806,0,c j stuart unfortunately yet still without lic...
20867,1,annyo84 will shall meet somewhere utah indiana
...,...,...
32725,2,back best party ever
2964,1,quietjack probably going thing
14695,0,lisatayl stay great mother gave cold flu left ...
1409,0,god piss always enter comp amp never win i sor...


In [114]:
X_train=train_data['content'].values
y_train=train_data['sentiment'].values

x_test=test_data['content'].values
y_test=test_data['sentiment'].values

In [115]:
Vectorizer=CountVectorizer()

X_train_bow=Vectorizer.fit_transform(X_train)
x_test_bow=Vectorizer.transform(x_test)

In [116]:
train_df=pd.DataFrame(X_train_bow.toarray())
train_df['label']=y_train
train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20988,20989,20990,20991,20992,20993,20994,20995,20996,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [117]:
from sklearn.metrics import accuracy_score,classification_report
from xgboost import XGBClassifier


In [ ]:
    model = model = XGBClassifier(
        objective="multi:softprob",
        num_class=3,
        eval_metric="mlogloss",
        n_estimators=400,
        max_depth=6,
        learning_rate=0.08,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )



    model.fit(X_train_bow,y_train)
    y_predict=model.predict(x_test_bow)

    accuracy=accuracy_score(y_test,y_predict)
    classification_reports=classification_report(y_test,y_predict)

    print("accuracy = ",accuracy)
    print("classification_report = ",classification_reports)

accuracy =  0.6160589060308556
classification_report =                precision    recall  f1-score   support

           0       0.71      0.46      0.56      1617
           1       0.58      0.79      0.67      2549
           2       0.63      0.49      0.55      1538

    accuracy                           0.62      5704
   macro avg       0.64      0.58      0.59      5704
weighted avg       0.63      0.62      0.61      5704

